#Despliegue de un Modelo de Clasificación con FastAPI usando el Dataset de Vinos

###Laura Vanessa Tamayo Pérez
```
Codigo 202422590
```


Link del dataset: https://scikit-learn.org/stable/datasets/toy_dataset.html#wine-dataset


In [27]:
#Instalación de la api
!pip install fastapi uvicorn nest_asyncio pyngrok joblib scikit-learn

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import joblib

# Se carga el dataset de wine sobre vinos
data = load_wine()
X = data.data
y = data.target

# se dividenn lo datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# se hace un escalado
# Aunque al utilizar un modelo de random Forest no es necesario hacer el escalamiento de los datos,
#llega a ser una buena práctica puesto que permite guardar el preprocesamiento para un entorno productivo
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenamiento del modelo
model = RandomForestClassifier(n_estimators=120, random_state=42)
model.fit(X_train_scaled, y_train)

#Se evalua el modelo
y_pred = model.predict(X_test)
print(f"Precisión del modelo: {accuracy_score(y_test, y_pred):.2f}")

#Se evalua el modelo escalado
y_pred = model.predict(X_test_scaled)
print(f"Precisión del modelo escalado: {accuracy_score(y_test, y_pred):.2f}")

# Guardar modelo y escalador
joblib.dump(model, 'modelo_clasificacion.pkl')
joblib.dump(scaler, 'escalador.pkl')


Precisión del modelo: 0.33
Precisión del modelo escalado: 1.00


['escalador.pkl']

Se puede observar que el modelo sin escalar llega a tener una precisión del 0.33, siendo algo bajo por lo que se decidió a trabajar con el modelo escalado a fin de tener unas mejores predicciones para este ejercicio.

In [51]:
# En este caso se plantea el uso de ngrok a fin de poder crear un túnel
#que sea público y accesible desde Internet y permita el acceso a la API de navegadores
#a las diferentes solicitudes, porr lo que se entrelaza con el token
!ngrok config add-authtoken 2ueVSZNb0UWHvJzAkPQ15eexe8A_5v3MHq3pins3oQ3dveTMB

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [52]:
import numpy as np

# Revisar clases únicas del modelo
print(model.classes_)


[0 1 2]


In [53]:
import logging

# Eliminar handlers anteriores si ya se ejecutó antes
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Configurar logging manualmente
logging.basicConfig(
    filename="log_api_vino_v2.log",
    filemode="a",  # 'a' para agregar, 'w' para sobrescribir
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

print("✅ Logging reconfigurado. Archivo: log_api_vino_v2.log")


✅ Logging reconfigurado. Archivo: log_api_vino_v2.log


In [54]:
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import joblib
import numpy as np
import logging

#requerido para ejecutar FastAPI en Colab
nest_asyncio.apply()

#cargar modelo guardado y escalador
modelo = joblib.load("modelo_clasificacion.pkl")
scaler = joblib.load("escalador.pkl")

#se inicializa FastAPI
app = FastAPI(title="API de Clasificación de vinos")

#definición de entradas de datos
class WineData(BaseModel):
    alcohol: float
    malic_acid: float
    ash: float
    alcalinity_of_ash: float
    magnesium: float
    total_phenols: float
    flavanoids: float
    nonflavanoid_phenols: float
    proanthocyanins: float
    color_intensity: float
    hue: float
    od280_od315_of_diluted_wines: float
    proline: float

# Endpoint para predicción
@app.post("/predecir/")
def predecir_vino(datos: WineData):
    entrada = np.array([[valor for valor in datos.model_dump().values()]])
    entrada_esc = scaler.transform(entrada)
    pred = modelo.predict(entrada_esc)[0]
    logging.info(f"Entrada: {datos.model_dump()} → Predicción: {float(pred)}")
    return {"prediccion": float(pred)}

#Endpoint a la raíz a fin de no aparecer el 404 Not Found
@app.get("/")
def read_root():
    return {"mensaje": "¡Bienvenido a la API de clasificación de vinos! Ve a /docs para usarla."}

# Crear túnel público
public_url = ngrok.connect(8000)
print(f"🌐 API disponible en: {public_url}/docs")

# Ejecutar servidor
uvicorn.run(app, host="0.0.0.0", port=8000)

✅ Logging reconfigurado. Archivo: log_api_vino_v2.log
🌐 API disponible en: NgrokTunnel: "https://0908-34-150-163-63.ngrok-free.app" -> "http://localhost:8000"/docs


INFO:     Started server process [276]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     191.156.159.26:0 - "GET / HTTP/1.1" 200 OK
INFO:     191.156.150.60:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     191.156.151.138:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     191.156.151.138:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /predecir/ HTTP/1.1" 422 Unprocessable Entity
INFO:     54.86.50.139:0 - "POST /predecir/ HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /predecir/ HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /predecir/ HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /predecir/ HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /predecir/ HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /predecir/ HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /predecir/ HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /predecir/ HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /predecir/ HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /predecir/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [276]


In [49]:
# Ver las últimas líneas del log
!tail log_api_vino_v2.log


tail: cannot open 'log_api_vino_v2.log' for reading: No such file or directory
